In [1]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
import pandas as pd
import numpy as np
from joblib import load, dump
import csv as cv
from math import exp

In [2]:
df = pd.read_csv('TestOracleRegression.csv')
data = df.values
X = data[:, :-1]
y = data[:, -1]

model = MLPRegressor(hidden_layer_sizes=(2))#2 idden layers with 2 neurons per layer
model.fit(X, y)

C:\Users\Arnab\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=2)

In [3]:
dump(model, 'Model/dnn_model.joblib')

['Model/dnn_model.joblib']

In [171]:
model.predict([[1,1,1,1]])

array([-0.18909288])

In [4]:
model.coefs_

[array([[-0.96683328,  0.2835481 ],
        [ 0.9882057 ,  0.1065518 ],
        [ 0.57472258, -0.41217302]]),
 array([[ 1.27865487],
        [-0.64427366]])]

In [174]:
model.n_outputs_

1

In [5]:
model.intercepts_


[array([-0.33765812, -0.89853701]), array([1.22110526])]

In [6]:
class ConvertDNN2logic:
    
    def __init__(self, image_data=False):
        with open('param_dict.csv') as csv_file:
            reader = cv.reader(csv_file)
            self.paramDict = dict(reader) 
        if self.paramDict['multi_label'] == 'True':
            self.dnn = load('Model/dnn_model_multi.joblib') 
        else:
            self.dnn = load('Model/dnn_model.joblib')
        self.df = pd.read_csv('Oracle.csv')
        self.no_of_params = int(self.paramDict['no_of_params'])    
        self.no_of_hidden_layer = len(self.dnn.coefs_) -1
        self.num_weight_vector = len(self.dnn.coefs_)
        #self.no_of_layer = len(self.dnn.linears)
        self.no_of_class = self.dnn.n_outputs_
        
        
    def remove_exponent(self, value):
        if 'e-' in value:
            decial = value.split('e')
            ret_val = format(((float(decial[0]))*(10**int(decial[1]))), '.5f')
            return ret_val
        else: 
            return value

    def funcDNN2logic(self):
        with open('feNameType.csv') as csv_file:
            reader = cv.reader(csv_file)
            feName_type = dict(reader)
        f=open('DNNSmt1.smt2', 'w')
        f.write(';Input layer neurons \n')
        for i in range(0, self.no_of_params):
            f.write(';-----------'+str(i)+'th parameter----------------- \n')
            #Initializing input features
            for j in range(0, self.dnn.n_features_in_):
                f.write('(declare-fun '+self.df.columns.values[j]+str(i)+' ()')
                if('int' in str(self.df.dtypes[j])):
                    f.write(' Int) \n')
                else:
                    f.write(' Real) \n')
                #f.write('(assert (and (>= '+self.df.columns.values[j]+str(i)+' 0) (<= '+self.df.columns.values[j]+str(i)
                #    +' 1))) \n')
                
                            
            #Initializing hidden layer neurons
            for j in range(0, self.num_weight_vector-1):
                for k in range(0, len(self.dnn.coefs_[j][0])):
                    f.write('(declare-fun nron'+str(j)+str(k)+str(i)+' () Real) \n')
                    f.write('(declare-fun tmp'+str(j)+str(k)+str(i)+' () Real) \n')
            #Initializing output neurons
            for j in range(0, self.no_of_class):
                f.write('(declare-fun y'+str(j)+str(i)+' () Real) \n')
                f.write('(declare-fun tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+' () Real) \n')
            #Initializing extra variables needed for encoding argmax 
            '''
            for j in range(0, self.no_of_class):
                for k in range(0, self.no_of_class):
                    f.write('(declare-fun d'+str(j)+str(k)+str(i)+' () Int) \n')
            '''
            if(self.paramDict['multi_label'] == 'FALSE'):
                if self.paramDict['regression'] == 'no':
                    f.write('(declare-fun Class'+str(i)+' () Int) \n')
                else:
                    f.write('(declare-fun Class' + str(i) + ' () Real) \n')
                    #f.write('(assert (and (>= Class'+str(i)+' 0) (<= Class'+str(i)+' 1))) \n')

            else:
                for j in range(0, self.no_of_class):
                    class_name = self.df.columns.values[self.df.shape[1]-self.no_of_class+j]
                    f.write('(declare-fun '+class_name+str(i)+' () Int) \n')
                    f.write('(assert (and (>= '+class_name+str(i)+' 0) (<= '+class_name+str(i)+' 1))) \n')
    
        f.write('(define-fun absoluteInt ((x Int)) Int \n')
        f.write('  (ite (>= x 0) x (- x))) \n')
        f.write('(define-fun absoluteReal ((x Real)) Real \n')
        f.write('  (ite (>= x 0) x (- x))) \n')  

    
        for i in range(0, self.no_of_params):
            f.write(';-----------'+str(i)+'th parameter----------------- \n')
            f.write('\n ;Encoding the hidden layer neurons \n')
            for j in range(0, self.no_of_hidden_layer):
                for k in range(0, len(self.dnn.coefs_[j][0])):
                    f.write('(assert (= ')
                    f.write('tmp'+str(j)+str(k)+str(i)+' (+')
                    for l in range(0, len(self.dnn.coefs_[j])):
                        #temp_val = round(float(self.dnn.linears[j].weight[k][l]), 5)
                        temp_val = format(self.dnn.coefs_[j][l][k], '.5f')
                        if('e' in str(temp_val)):
                            temp_val = self.remove_exponent(str(temp_val))
                        if j == 0:
                            f.write('(* '+self.df.columns.values[l]+str(i)+' '+str(temp_val)+')')
                        else:
                            f.write('(* nron'+str(j)+str(k)+str(i)+' '+str(temp_val)+')')
                    #temp_bias = round(float(self.dnn.linears[j].bias[k]), 5)
                    temp_bias = format(self.dnn.intercepts_[j][k], '.5f')
                    if('e' in str(temp_bias)):
                        temp_bias = self.remove_exponent(str(temp_bias))
                    f.write(str(temp_bias)+'))) \n')

                    f.write('(assert (=> (> tmp'+str(j)+str(k)+str(i)+' 0) (= nron'+str(j)+str(k)+str(i)+
                           ' tmp'+str(j)+str(k)+str(i)+'))) \n')
                    f.write('(assert (=> (<= tmp'+str(j)+str(k)+str(i)+' 0) (= nron'+str(j)+str(k)+str(i)+
                           ' 0))) \n')
            
            f.write('\n ;Encoding the output layer neuron \n')           
            for j in range(0, self.dnn.n_outputs_):
                f.write('(assert (= tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+' (+ ')
                for k in range(0, len(self.dnn.coefs_[self.no_of_hidden_layer-1][0])):
                    #temp_val = round(float( self.dnn.linears[self.no_of_layer-1].weight[j][k]), 5)
                    temp_val = format(self.dnn.coefs_[self.no_of_hidden_layer][k][j], '.5f')
                    if('e' in str(temp_val)):
                        temp_val = self.remove_exponent(str(temp_val))
                    f.write('(* nron'+str(self.no_of_hidden_layer-1)+str(k)+str(i)+' '+str(temp_val)+')')
                #temp_bias = round(float(self.dnn.linears[self.no_of_layer-1].bias[j]), 5)
                temp_bias = format(self.dnn.intercepts_[self.no_of_hidden_layer][j], '.5f')
                if('e' in str(temp_bias)):
                    temp_bias = self.remove_exponent(str(temp_bias))
                f.write(' '+str(temp_bias)+'))) \n')
                f.write('(assert (= y'+str(j)+str(i)+
                           ' tmp'+str(self.no_of_hidden_layer)+str(j)+str(i)+')) \n')
            f.write('(assert (= Class'+str(i)+' y0'+str(i)+'))\n')   
                          
           
        f.write('(assert (= Class0 1))\n')
        f.write('(check-sat) \n(get-model)\n')
        f.close()
        

In [7]:
dnn2logic = ConvertDNN2logic()
dnn2logic.funcDNN2logic()

In [31]:
model.n_features_in_

4

In [86]:
with open('param_dict.csv') as csv_file:
    reader = cv.reader(csv_file)
    paramDict = dict(reader)

In [83]:
df = pd.read_csv('param_dict.csv')

In [84]:
df.to_csv('param_dict.csv', index=False, header=True)

In [85]:
df

,max_samples,1000
0,regression,no
1,bound_cex,no
2,upper_bound,1
3,lower_bound,0
4,assumeData,50
5,deadlines,1000
6,white_box_model,Decision tree
7,no_EPOCHS,50
8,no_of_params,2
9,mul_cex_opt,FALSE


In [150]:
-0.6483+0.744-0.10489+0.6888-1.12857798

-0.44896798000000004

In [161]:
exp(-0.893)

0.40942563159841006

In [158]:
exp(-1.2417)+ exp(-0.893) + exp(0.8404)

3.0156120231961756

In [162]:
0.4094/3.01561

0.13576026077642664

In [181]:
945457877.0/5000000000.0

0.1890915754

In [ ]:
array[]